In [3]:
from __future__ import print_function
from openpathsampling.experimental.simstore.sql_backend import universal_schema, universal_sql_meta
from openpathsampling.experimental.simstore import SQLStorageBackend
st = SQLStorageBackend(":memory:", mode='w')
import sqlalchemy as sql

In [2]:
import logging
import sys

import collections

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

In [3]:
schema = {
    'samples': [('replica', 'int'),
                ('ensemble', 'uuid'),
                ('trajectory', 'uuid')]
}

In [4]:
new_schema = {
    'snapshot0': [('filename', 'str'),
                  ('index', 'int')]
}

In [5]:
table_to_class = {'samples': tuple, 'snapshot0': tuple}

In [6]:
st.register_schema(schema, table_to_class)

2020-05-19 10:03:06,886 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table samples


In [7]:
st.register_schema(new_schema, table_to_class)

2020-05-19 10:03:06,904 - openpathsampling.experimental.storage.sql_backend - INFO - Add schema table snapshot0


In [8]:
# practical storage test:
# 1. Run 1000 steps MD (AD)
# 2. (time) open storage (w); store trajectory; close storage
# 3. (time) open storage (r); load trajectory
# 4. (time) calculate a CV on the trajectory

In [9]:
st.engine.table_names()

['metadata', 'samples', 'schema', 'snapshot0', 'tables', 'uuid']

In [10]:
with st.engine.connect() as conn:
    result = conn.execute(st.metadata.tables['tables'].select())
    for row in result:
        print(row)

('samples', 0, 'builtins', 'tuple')
('snapshot0', 1, 'builtins', 'tuple')


In [11]:
sample_list = [(0, 'ens1', 'traj1'),
               (1, 'ens2', 'traj2'),
               (0, 'ens1', 'traj2')]
sample_dict = [{'replica': s[0], 'ensemble': s[1], 'trajectory': s[2]}  for s in sample_list]
for s in sample_dict:
    s.update({'uuid': str(hex(hash(str(sample_dict))))})

In [12]:
sample_dict

[{'replica': 0,
  'ensemble': 'ens1',
  'trajectory': 'traj1',
  'uuid': '-0x286ab060e638caca'},
 {'replica': 1,
  'ensemble': 'ens2',
  'trajectory': 'traj2',
  'uuid': '-0x35f7d5f63b434af1'},
 {'replica': 0,
  'ensemble': 'ens1',
  'trajectory': 'traj2',
  'uuid': '-0x6ab32c80ae3fd51f'}]

In [13]:
st.add_to_table('samples', sample_dict)

In [14]:
with st.engine.connect() as conn:
    result = conn.execute(st.metadata.tables['samples'].select())
    for row in result:
        print(row)

(1, '-0x286ab060e638caca', 0, 'ens1', 'traj1')
(2, '-0x35f7d5f63b434af1', 1, 'ens2', 'traj2')
(3, '-0x6ab32c80ae3fd51f', 0, 'ens1', 'traj2')


In [15]:
with st.engine.connect() as conn:
    result = conn.execute(st.metadata.tables['uuid'].select())
    for row in result:
        print(row)

('-0x286ab060e638caca', 0, 1)
('-0x35f7d5f63b434af1', 0, 2)
('-0x6ab32c80ae3fd51f', 0, 3)


In [16]:
uuids = [sample_dict[0]['uuid'], sample_dict[1]['uuid']]

In [17]:
uuid_table = st.metadata.tables['uuid']
uuid_or_stmt = sql.or_(*(uuid_table.c.uuid == uuid
                         for uuid in uuids))
uuid_sel = uuid_table.select(uuid_or_stmt)

In [18]:
print(uuid_sel)

SELECT uuid.uuid, uuid."table", uuid."row" 
FROM uuid 
WHERE uuid.uuid = ? OR uuid.uuid = ?


In [19]:
print(uuids)

['-0x286ab060e638caca', '-0x35f7d5f63b434af1']


In [20]:
uuid_rows = st.load_uuids_table(uuids)

2020-05-19 10:03:07,236 - openpathsampling.experimental.storage.sql_backend - DEBUG - Looking for 2 UUIDs
2020-05-19 10:03:07,239 - openpathsampling.experimental.storage.sql_backend - DEBUG - New block of 2 UUIDs
2020-05-19 10:03:07,241 - openpathsampling.experimental.storage.sql_backend - DEBUG - Found 2 UUIDs


In [21]:
st.load_table_data(uuid_rows)

[(1, '-0x286ab060e638caca', 0, 'ens1', 'traj1'),
 (2, '-0x35f7d5f63b434af1', 1, 'ens2', 'traj2')]

In [22]:
results = _

In [23]:
results[0].uuid

'-0x286ab060e638caca'

In [24]:
results[0].uuid

'-0x286ab060e638caca'

In [25]:
table = st.metadata.tables['tables']
with st.engine.connect() as conn:
    results = list(conn.execute(table.select()))
results

[('samples', 0, 'builtins', 'tuple'), ('snapshot0', 1, 'builtins', 'tuple')]

In [26]:
# TODO: add support for table metadata (add a metadata table, with table metadata stored therein)